In [1]:
!pip3 install whisper-timestamped

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 901.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.5/770.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 25.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvid

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import json
import IPython.display as ipd

import librosa
import librosa.display

import soundfile as sf
import wave
import torch
import torchaudio

from google.colab import drive

from scipy.io.wavfile import write

import whisper_timestamped as whisper

import utils as u

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [3]:
# mount google drive
# change data_path as needed to wherever you want to put output
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/megaphone/audio_samples"

Mounted at /content/drive


In [4]:
# these are the pitch adjustments that will be made, in semitones
d_pitches = [-2,-1,0,1,2]
# these are the percentages that the track or word will be slowed
p_slowers = [0.15, 0.1, 0.08, 0.05, 0.03]
# name of the file you are using
file_path = "megaphone_test_audio_small.mp3"
# common prefix for all saved files
save_prefix = "7_24_v3"

In [5]:
save_path = data_path+"/"+save_prefix

In [6]:
save_path

'/content/drive/MyDrive/megaphone/audio_samples/7_24_v3'

In [7]:
# load the audio sample
y,sr = librosa.load(file_path)

In [8]:
# run the audio through whisper to get the words and timestamps (start + end times of words)

audio = whisper.load_audio(file_path)

model = whisper.load_model("tiny", device="cpu")

result = whisper.transcribe(model, audio, language="en")

result_formatted = json.dumps(result, indent = 2, ensure_ascii = False)

result_dict = json.loads(result_formatted)
print(result_dict)

100%|█████████████████████████████████████| 72.1M/72.1M [00:11<00:00, 6.68MiB/s]
100%|██████████| 337/337 [00:01<00:00, 191.99frames/s]

{'text': ' Hello there and welcome to improve your voice.', 'segments': [{'id': 0, 'seek': 0, 'start': 0.08, 'end': 3.2, 'text': ' Hello there and welcome to improve your voice.', 'tokens': [50364, 2425, 456, 293, 2928, 281, 3470, 428, 3177, 13, 50524], 'temperature': 0.0, 'avg_logprob': -0.40446317195892334, 'compression_ratio': 0.8518518518518519, 'no_speech_prob': 0.020983437076210976, 'confidence': 0.846, 'words': [{'text': 'Hello', 'start': 0.08, 'end': 0.5, 'confidence': 0.872}, {'text': 'there', 'start': 0.5, 'end': 0.84, 'confidence': 0.967}, {'text': 'and', 'start': 0.84, 'end': 1.28, 'confidence': 0.827}, {'text': 'welcome', 'start': 1.28, 'end': 1.66, 'confidence': 0.976}, {'text': 'to', 'start': 1.66, 'end': 2.06, 'confidence': 0.991}, {'text': 'improve', 'start': 2.06, 'end': 2.56, 'confidence': 0.396}, {'text': 'your', 'start': 2.56, 'end': 2.92, 'confidence': 0.985}, {'text': 'voice.', 'start': 2.92, 'end': 3.2, 'confidence': 0.997}]}], 'language': 'en'}


In [9]:
# sometimes the words appear in the 0th index, sometimes in 1... idk why
try:
  result_dict_indexed = result_dict["segments"][1]["words"]
except:
  result_dict_indexed = result_dict["segments"][0]["words"]

In [10]:
# get words and start times
start_times = []
words = []
for word in result_dict_indexed:
  start_times.append(word["start"])
  words.append(word["text"])

In [11]:
# names of directories
file_partial_names = ["all_slowed",
              "alt_speed",
              "random_word_slower_sample",
              "chosen_word_slower_sample",
              "all_pitched_down",
              "alt_pitch",
              "random_word_pitched_down_sample",
              "chosen_word_pitched_down_sample",
              "all_slow_rand_pitched",
              "all_slow_chosen_pitched",
              "all_pitched_rand_slow",
              "all_pitched_chosen_slow",
              "chosen_word_slowed_and_pitched",
              "chosen_word_pitched_and_slowed"
              ]

directories = [data_path + "/" + save_prefix + "/" + x for x in file_partial_names]

In [12]:
# write files with this naming convetion
def write1():
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+  "$all_slowed.wav", sr, all_slowed)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ "$alt_speed.wav", sr, alt_speed)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ "$random_word_slower_sample.wav", sr, random_word_slower_sample)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ "$chosen_word_slower_sample.wav", sr, chosen_word_slower_sample)

def write2():
  write(save_prefix + file_path.split(".")[0] + str(d_pitch)+ "$all_pitched_down.wav", sr, all_pitched_down)
  write(save_prefix + file_path.split(".")[0] + str(d_pitch)+ "$alt_pitch.wav", sr, alt_pitch)
  write(save_prefix + file_path.split(".")[0] + str(d_pitch)+ "$random_word_pitched_down_sample.wav", sr, random_word_pitched_down_sample)
  write(save_prefix + file_path.split(".")[0] + str(d_pitch)+ "$chosen_word_pitched_down_sample.wav", sr, chosen_word_pitched_down_sample)

def write3():
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ str(d_pitch)+ "$all_slow_rand_pitched.wav", sr, all_slow_rand_pitched)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ str(d_pitch)+ "$all_slow_chosen_pitched.wav", sr, all_slow_chosen_pitched)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ str(d_pitch)+ "$all_pitched_rand_slow.wav", sr, all_pitched_rand_slow)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ str(d_pitch)+ "$all_pitched_chosen_slow.wav", sr, all_pitched_chosen_slow)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ str(d_pitch)+ "$chosen_word_slowed_and_pitched.wav", sr, chosen_word_slowed_pitched)
  write(save_prefix + file_path.split(".")[0] + str(p_slow)+ str(d_pitch)+ "$chosen_word_pitched_and_slowed.wav", sr, chosen_word_pitch_slowed)

In [13]:
# make directories in google drive for each sample
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}

!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/all_slowed
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/alt_speed
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/random_word_slower_sample
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/chosen_word_slower_sample

!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/all_pitched_down
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/alt_pitch
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/random_word_pitched_down_sample
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/chosen_word_pitched_down_sample

!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/all_slow_rand_pitched
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/all_slow_chosen_pitched
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/all_pitched_rand_slow
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/all_pitched_chosen_slow
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/chosen_word_slowed_and_pitched
!mkdir -p /content/drive/MyDrive/megaphone/audio_samples/{save_prefix}/chosen_word_pitched_and_slowed

In [14]:
# generate and save all slow-only alterations
for p_slow in p_slowers:
  all_slowed = u.all_slow(y, p_slow)
  alt_speed = u.alternate_speed(y, p_slow, start_times)
  random_word_slower_sample = u.random_word_slow(y, p_slow, start_times)
  chosen_word_slower_sample = u.chosen_word_slower(y, p_slow, start_times, words, pos = "verb")
  write1()

# generate and save all pitch-only alterations
for d_pitch in d_pitches:
  all_pitched_down = u.all_pitched_down(y, d_pitch)
  alt_pitch = u.alternate_pitch(y, d_pitch, start_times)
  random_word_pitched_down_sample = u.random_word_pitched_down(y, d_pitch, start_times)
  chosen_word_pitched_down_sample = u.chosen_word_pitched_down(y, d_pitch, start_times, words, pos = "verb")
  write2()


# generate and save all slow and pitch alterations
for p_slow in p_slowers:
  for d_pitch in d_pitches:
    all_slow_rand_pitched = u.all_slow_random_word_pitched_down(y, p_slow, d_pitch, start_times)
    all_slow_chosen_pitched = u.all_slow_chosen_word_pitched_down(y, p_slow, d_pitch, start_times, words, "verb")
    try:
      all_pitched_rand_slow = u.all_pitched_down_random_slowed(y, d_pitch, p_slow, start_times)
    except:
      pass
    try:
      all_pitched_chosen_slow = u.all_pitched_down_chosen_word_slowed(y, d_pitch, p_slow, start_times, words, "verb")
    except:
      pass
    chosen_word_slowed_pitched = u.chosen_word_slowed_and_pitched(y, d_pitch, p_slow, start_times, words, "verb")
    chosen_word_pitch_slowed = u.chosen_word_pitched_and_slowed(y, d_pitch, p_slow, start_times, words, "verb")
    write3()

0
1
2
3
4
5
6
8


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
in verb clause
0
1
2
3
4
5
6
8
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
in verb clause
0
1
2
3
4
5
6
8
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
in verb clause
0
1
2
3
4
5
6
8
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
in verb clause
0
1
2
3
4
5
6
8
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
i

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


0
out of while
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
7
out of while
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
1
out of while
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3


...

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
6
out of while
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
in verb clause
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
7
out of while
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
Hello
verb
there
verb
and
verb
in verb clause
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3
[('Hello', 'NNP'), ('there', 'EX'), ('and', 'CC'), ('welcome', 'VB'), ('to', 'TO'), ('improve', 'VB'), ('your', 'PRP$'), ('voice.', 'NN')]
verb
pos verb
in verb
3


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
import shutil

# move files from colab to drive
def move_wav_files_to_drive(source_folder, destination_folder):
    import os
    source_folder_path = source_folder
    destination_folder_path = destination_folder

    # Get a list of all files in the source folder
    files = os.listdir(source_folder_path)

    # Iterate through files and move WAV files to the destination folder
    for file in files:
        if file.lower().endswith(".wav"):
          #print(file.split("$"))
          to_match = file.split("$")[1].split(".")[0]
          for dir in os.listdir(destination_folder):
            if to_match == dir:
              try:
                shutil.move(os.path.join(source_folder_path, file), os.path.join(destination_folder_path, dir, file))
                print(f"Moved {file} to {destination_folder}")
              except Exception as e:
                print(f"Error moving {file}: {e}")


In [16]:
# move files from colab to drive
import os
current_dir = os.getcwd()
move_wav_files_to_drive(current_dir, save_path)

Moved 7_24_v3megaphone_test_audio_small2$random_word_pitched_down_sample.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.03-1$all_slow_rand_pitched.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.1-1$all_slow_chosen_pitched.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.1$random_word_slower_sample.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.1-2$chosen_word_pitched_and_slowed.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.032$chosen_word_pitched_and_slowed.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.031$all_slow_chosen_pitched.wav to /content/drive/MyDrive/megaphone/audio_samples/7_24_v3
Moved 7_24_v3megaphone_test_audio_small0.152$all_pitched_chosen_sl